In [1]:
from scripts.scraping import scrape_symbols, scrape_financial, download_data
import os

In [2]:
FIRST_RUN = False
cotation_url = ['https://www.slickcharts.com/sp500']

In [3]:
if FIRST_RUN:
    if 'data' not in os.listdir('..'):
        os.mkdir('../data')
    if 'symbols.txt' not in os.listdir('../data'):
        with open('../data/symbols.txt', 'w') as f:
            pass
    if 'financials' not in os.listdir('..'):
        os.mkdir('../data/financials')
    if 'links.txt' not in os.listdir('../data'):
        with open('../data/links.txt', 'w') as f:
            pass
    symbols = scrape_symbols(cotation_url, '../data/symbols.txt')
else:
    with open('../data/symbols.txt', 'r') as f:
        symbols = f.readlines()
        symbols = [symbol.strip() for symbol in symbols]

In [4]:
links = scrape_financial(symbols, progress_file='../data/links.txt')

Scraping financial data:   1%|▏         | 7/477 [00:36<36:58,  4.72s/it]

In [ ]:
download_data('../data/links.txt')

In [20]:
#Models
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tqdm import tqdm

In [23]:
def dataset_preparation():
    data = pd.read_csv('../data/financials.csv')
    dict_df = {}
    for symbol in tqdm(data['Symbol'].unique(), desc='Processing data'):
        dict_df[symbol] = data[data['Symbol'] == symbol][['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    return dict_df

dict_df = dataset_preparation()

Processing data: 100%|██████████| 501/501 [01:21<00:00,  6.15it/s]


In [24]:
#Merge all dataframes on the date column and add suffixes to the columns
merged_df = pd.DataFrame()
for symbol in tqdm(dict_df.keys(), desc='Merging dataframes'):
    if merged_df.empty:
        merged_df = dict_df[symbol]
    else:
        merged_df = pd.merge(merged_df, dict_df[symbol], on='Date', suffixes=('', f'_{symbol}'))

,Symbol,Date,Open,High,Low,Close,Adj Close,Volume
10813,AAPL,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099192,469033600.0
10814,AAPL,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094017,175884800.0
10815,AAPL,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087117,105728000.0
10816,AAPL,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089273,86441600.0
10817,AAPL,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091861,73449600.0
...,...,...,...,...,...,...,...,...
21735,AAPL,2024-04-11,168.339996,175.460007,168.160004,175.039993,175.039993,91070300.0
21736,AAPL,2024-04-12,174.259995,178.360001,174.210007,176.550003,176.550003,101593300.0
21737,AAPL,2024-04-15,175.360001,176.630005,172.500000,172.690002,172.690002,73531800.0
21738,AAPL,2024-04-16,171.750000,173.759995,168.270004,169.380005,169.380005,73711200.0
